# Author Name Recon Part 2

Following part 2, there are three main cases of ambiguity that can be resolved by affiliations and other methods, we do so here using a number of strategies. Given the disjointed nature of the affiliations data, I will also use this process to iteratively consolidate the two sets of affiliations for common 

This part combines

## Three cases of ambiguity:
1. same name, different person
   - is in same time period -> require affiliations
   - not in same time period -> resolve with significant significant publishing year barrier
2. similar name or alias, same person
   - has multiple resolution from no middle name to has middle name: require affiliations
   - spelling error possibilities. Direct correction in data - require research on individual articles for resolution, restricted to this data set only
3. has multiple resolutions from contraction: require affiliations
   - generate list of contractions and associated possibilities if full name exists


Notes
We only care about authors of articles (any not misc or reviews) between 1940 and 2020
We can only process those articles with affiliations recorded or explicitly set to none

In [2]:
import pandas as pd
# from unidecode import unidecode
import re
from datetime import date
import json
# import numpy as np
import string
import time
# from fuzzywuzzy import fuzz 
# from fuzzywuzzy import process 
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [3]:
base_path="/Users/sijiawu/Work/Thesis/Data/Affiliations/"

In [4]:
with open(base_path+"author_proc_reduced.json") as f: 
    data = f.read() 

proc_auths_all = json.loads(data) 

In [6]:
a1=[]
a2=[]
a3=[]
reduced={}
condense={}
f=[]
d=[]
o=[]
e=[]
k=0
for i in proc_auths_all.keys():
    try:
        if (proc_auths_all[i]["content_type"]!="MISC") & (proc_auths_all[i]["content_type"]!="Review"):
            if (bool(proc_auths_all[i]["authors"]['0']['affs']) & (proc_auths_all[i]["year"]>=1940)): #if the affiliations are found
                reduced[i]=proc_auths_all[i]
                for j in proc_auths_all[i]['authors'].keys():
                    temp=proc_auths_all[i]['authors'][j]
                    a1.append(temp['a1'])
                    a2.append(temp['a2'])
                    a3.append(temp['a3'])
                    
                    mains=[]
                    for l in temp["affs"].keys():
                        mains.append(temp["affs"][l]["aff_main_final"])
                    reduced[i]['authors'][j]["affs"]=mains    
                    condense[k]={"a1":temp['a1'], "a2": temp["a2"], "a3": temp["a3"], "last": temp['a1'].split(" ")[-1],"affs": mains, "year":proc_auths_all[i]["year"], "url": i}
                    k=k+1
            else:
                if (proc_auths_all[i]["year"]>=1940):
                    d.append(i)
        else:
            o.append(i)
    except:
        e.append(i)

# Approach
1. At a1, problem of are these two authors the same person despite having the same spelling?
2. At a2, are these two the same person despite having different spelling originally?
3. At a3, are these two the same person despite having different spelling originally?
4. At a3, if author names only result in a unique author




In [7]:
proc_auth_flat=pd.DataFrame(condense).transpose()


In [10]:
proc_auth_flat.head()

,a1,a2,a3,last,affs,year,url
0,nicola gennaioli,n. gennaioli,n. gennaioli,gennaioli,"[Pompeu Fabra University, Centre De Recerca En Economia Internacional (Crei, Barcelona), Centre For Economic Policy Research (Cepr, England)]",2010,https://www.jstor.org/stable/40961010
1,andrei shleifer,a. shleifer,a. shleifer,shleifer,[Harvard University],2010,https://www.jstor.org/stable/40961010
2,dean p. foster,d. p. foster,d. foster,foster,[University Of Pennsylvania],2010,https://www.jstor.org/stable/40961011
3,h. p. young,h. p. young,h. young,young,"[University Of Oxford, Brookings Institution]",2010,https://www.jstor.org/stable/40961011
4,eric d. gould,e. d. gould,e. gould,gould,"[Hebrew University Of Jerusalem, Centre For Economic Policy Research (Cepr, England)]",2010,https://www.jstor.org/stable/40961012


Author names have been manipulated into three formats. There is variation in how the names are recorded by default so each round of contracting the name is a subset of the original. For example: 

- a1 - middle names contracted/OG (Henry J. Aaron => Henry J. Aaron or H. J. Aaron or H. Aaron), 
- a2 - first names contracted (above => H. J. Aaron or H. Aaron), 
- a3 - first letter of first name and last name (above => H. Aaron). 

In [13]:
uni=list(set(a1))
uni2=list(set(a2))
uni3=list(set(a3))
proc_auth_flat['a1_order']=None
proc_auth_flat['a2_order']=None
proc_auth_flat['a3_order']=None
for i in proc_auth_flat.index:
    index = uni.index(proc_auth_flat.loc[i, "a1"])
    index2 = uni2.index(proc_auth_flat.loc[i, "a2"])
    index3 = uni3.index(proc_auth_flat.loc[i, "a3"])

    proc_auth_flat.loc[i, 'a1_order']=index
    proc_auth_flat.loc[i, 'a2_order']=index2
    proc_auth_flat.loc[i, 'a3_order']=index3

In [14]:
proc_auth_flat_sorted=proc_auth_flat.sort_values(['last', 'a1', 'year'], ascending=[True, True, True])
proc_auth_flat_sorted.head()


,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order
38172,staff o. s. s. 125,s. o. s. s. 125,s. 125,125,[Harvard University],1970,https://www.jstor.org/stable/1815828,7289,12570,5277
38539,henry aaron,h. aaron,h. aaron,aaron,[University Of Maryland],1967,https://www.jstor.org/stable/1814407,3705,13984,12473
14593,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1909699,3705,13984,12473
38043,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1818280,3705,13984,12473
37430,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1974,https://www.jstor.org/stable/1816044,3705,13984,12473


In [23]:
proc_auth_flat_sorted.head(20)

,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order
38172,staff o. s. s. 125,s. o. s. s. 125,s. 125,125,[Harvard University],1970,https://www.jstor.org/stable/1815828,7289,12570,5277
38539,henry aaron,h. aaron,h. aaron,aaron,[University Of Maryland],1967,https://www.jstor.org/stable/1814407,3705,13984,12473
14593,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1909699,3705,13984,12473
38043,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1818280,3705,13984,12473
37430,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1974,https://www.jstor.org/stable/1816044,3705,13984,12473
14013,henry aaron,h. aaron,h. aaron,aaron,[Brookings Institution],1976,https://www.jstor.org/stable/1912735,3705,13984,12473
37049,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1976,https://www.jstor.org/stable/1817220,3705,13984,12473
34024,henry j. aaron,h. j. aaron,h. aaron,aaron,[Brookings Institution],1989,https://www.jstor.org/stable/1827721,3888,8586,12473
34041,henry j. aaron,h. j. aaron,h. aaron,aaron,[Brookings Institution],1989,https://www.jstor.org/stable/1827737,3888,8586,12473
17106,daniel aaronson,d. aaronson,d. aaronson,aaronson,[Federal Reserve Bank of Chicago],2011,https://www.jstor.org/stable/10.1086/662962,8357,7842,6986


In [26]:
case_1=proc_auth_flat_sorted[proc_auth_flat_sorted["last"]=="aaron"]

In [27]:
case_1

,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order
38539,henry aaron,h. aaron,h. aaron,aaron,[University Of Maryland],1967,https://www.jstor.org/stable/1814407,3705,13984,12473
14593,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1909699,3705,13984,12473
38043,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1818280,3705,13984,12473
37430,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1974,https://www.jstor.org/stable/1816044,3705,13984,12473
14013,henry aaron,h. aaron,h. aaron,aaron,[Brookings Institution],1976,https://www.jstor.org/stable/1912735,3705,13984,12473
37049,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1976,https://www.jstor.org/stable/1817220,3705,13984,12473
34024,henry j. aaron,h. j. aaron,h. aaron,aaron,[Brookings Institution],1989,https://www.jstor.org/stable/1827721,3888,8586,12473
34041,henry j. aaron,h. j. aaron,h. aaron,aaron,[Brookings Institution],1989,https://www.jstor.org/stable/1827737,3888,8586,12473


In [36]:
case_2=proc_auth_flat_sorted[proc_auth_flat_sorted["last"]=="wolf"]
case_2

,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order,consol_id_0,consol_id_1
3065,charles r. wolf,c. r. wolf,c. wolf,wolf,[Columbia University],1971,https://www.jstor.org/stable/1880705,13474,13787,1855,None,None
39642,charles wolf,c. wolf,c. wolf,wolf,[Rand Corporation],1955,https://www.jstor.org/stable/1821384,8776,2159,1855,None,None
2949,charles wolf,c. wolf,c. wolf,wolf,[Rand Corporation],1973,https://www.jstor.org/stable/1882036,8776,2159,1855,None,None
37600,charles wolf,c. wolf,c. wolf,wolf,[Rand Corporation],1973,https://www.jstor.org/stable/1817051,8776,2159,1855,None,None
12910,charles wolf,c. wolf,c. wolf,wolf,[Columbia University],1983,https://www.jstor.org/stable/1912161,8776,2159,1855,None,None
40423,h. d. wolf,h. d. wolf,h. wolf,wolf,[],1947,https://www.jstor.org/stable/1821173,12100,11431,6792,None,None
32883,holger c. wolf,h. c. wolf,h. wolf,wolf,[New York University],1993,https://www.jstor.org/stable/2117556,3997,4427,6792,None,None
10961,michael wolf,m. wolf,m. wolf,wolf,[Pompeu Fabra University],2001,https://www.jstor.org/stable/2692222,11468,552,475,None,None
10453,michael wolf,m. wolf,m. wolf,wolf,[University Of Zurich],2005,https://www.jstor.org/stable/3598821,11468,552,475,None,None
20695,thomas a. wolf,t. a. wolf,t. wolf,wolf,[International Institute Of Management Berlin],1973,https://www.jstor.org/stable/1837339,9244,7037,4014,9244,None


Ambiguity in author names - spelling

Suppose we group the smallest contraction, a3, eg: "h. aaron" by the index a3_order, "12373", and within this group compare a1 author name type "henry aaron or h. aaron" whatever has resulted from <first name> <contrated middle name> <last name> by checking how many a1_order indices occur in this a3_order group. 

-- cases of no need for affiliations.
1. If the a3_order corresponds to a single row of author names then this name is unique in this dataset.

-- potential need for affiliations
2. If the a1_order is the same for all author names in the a3_order group then this a3 name and it's associated a1 name is a unique author. 

-- a1 conflicts
Suppose there is a conflict, such that there are multiple a1 names for each a3 name:
Let's say "h. aaron" has "henry j. aaron" has four variations in a1: "henry j. aaron", "henry aaron", "h. j. aaron", "h. aaron", "hailey aaron".
1. Check first name variations. Only henry and henry should be checked.  
2. If hailey is unique then we asign the number to the consolidation.
3. Is henry j. aaron and henry aaron the same? check affiliations
4. is h. aaron for hailey or for henry? check affiliations

Ambiguity within the group
for 

Ambiguity accross time



In [114]:
from itertools import combinations
from itertools import chain

proc_auth_flat_sorted["consol_id_0"]=None
proc_auth_flat_sorted["consol_id_1"]=None

proc_auth_flat_sorted["a1_first"]=proc_auth_flat_sorted["a1"].str.split(" ").str[0]
proc_auth_flat_sorted["a1_split_count"]=proc_auth_flat_sorted["a1"].str.split(" ").str.len()

lm=proc_auth_flat_sorted

duplicate=0
contracts=0

round1=[]
unique_0=[]
unique_1=[]

order="a2_order"
back="a1_order"

stop=0

merge_pos=0

for i in range(len(uni3)):
    # print(uni3[i])
    sample=lm[lm["a3_order"]==i]
    if sample.shape[0]==1: #check if there is a one line unique entry, this implies no conflict in author names
        # print(sample["a1_order"].to_list()[0])
        unique_0.append(sample["a1_order"].to_list()[0])
        proc_auth_flat_sorted.loc[sample.index,"consol_id_0"]=sample["a1_order"].to_list()[0]
    else:
        unique_a1s=sample['a1_order'].unique()
        if len(unique_a1s)==1:
            unique_1.append(sample["a1_order"].to_list()[0])
            proc_auth_flat_sorted.loc[sample.index,"consol_id_0"]=sample["a1_order"].to_list()[0]
        else:
            round1.append(i)
            duplicate=duplicate+1
            
            print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

            # stop+=1
            # if stop>3:
            #     break

            a3_eq_a1=sample[(sample["a1"]==sample["a3"])] # anything that already looks like a3 at a1
            a2_eq_a1=sample[(sample["a1"]==sample["a2"])&(sample["a1"]!=sample["a3"])] #anything that already looks like a2 at a1 but not like a3
            a1_sims=sample[(sample["a1"]!=sample["a2"])&(sample["a1"]!=sample["a3"])] #any a1 values that are not a2 and or a3 already
            if ((a3_eq_a1.shape[0]+a2_eq_a1.shape[0]+a1_sims.shape[0])==(sample.shape[0]))==False:
                print(sample)
                print("There is an issue with the segementation")

            if a1_sims.shape[0]==0:
                contracts+=1

            a1_sims_unique=a1_sims['a1_order'].unique()
            combos=list(combinations(a1_sims_unique, 2))

            for k in combos:
                a=sample[sample["a1_order"]==k[0]]
                b=sample[sample["a1_order"]==k[1]]
                print(a[["a1", "a2","affs",]])
                print(b[["a1", "a2","affs",]])
                
                if a["a1_first"].values[0]==b["a1_first"].values[0]:

                    # check the middle initial if any
                    if ((a["a1_split_count"].values[0]>2)&(b["a1_split_count"].values[0]>2)):
                        if(a["a1"].values[0].split(" ")[1]!=b["a1"].values[0].split(" ")[1]):
                            continue
                        
                    target=k[0]
                    if len(a["a1"].values[0])<len(b["a1"].values[0]):
                        target=k[1]
                    merge_pos+=1
                    b_=[]
                    a_=[]
                    for l in b["affs"].values:
                        b_=[*l,*b_]
                    for l in a["affs"].values:
                        a_=[*l,*a_]
                    intersect=list(set(a_) & set(b_))
                    if len(intersect)>0:
                        temp=proc_auth_flat_sorted[(proc_auth_flat_sorted['a1_order']==k[0]) | (proc_auth_flat_sorted['a1_order']==k[1])]
                        proc_auth_flat_sorted.loc[temp.index,"consol_id_1"]=target
                print("###########################")

            a2_unique=a2_eq_a1['a1_order'].unique()
            for k in a2_unique:
                print("check the a2 variants that already exist in a1 against each a1 remainder")

            a3_unique=a3_eq_a1['a1_order'].unique()

            for k in a3_unique:
                print("check the a3 variants that already existed in a1 against the remainder")
            # if a3_eq_a1.shape[0]>0:
            #     print(a3_eq_a1[["a1", "a2","affs", "a1_order"]])
            # if a2_eq_a1.shape[0]>0:
            #     print(a2_eq_a1[["a1", "a2","affs", "a1_order"]])
            # if a1_sims.shape[0]>0:
            #     print(a1_sims[["a1", "a2","affs", "a1_order"]])
            # print(a1_sims[["a1", "a2","affs", "a1_order"]])
            # print(combos)
            # for j in combos:
            #     a=sample[sample["a1_order"]==j[0]]
            #     b=sample[sample["a1_order"]==j[1]]
                # print(str(b.shape)+ " "+ str(a.shape))
            # if len(lst)>2: 
                # print(len(unique_a1s))
                # print(sample[["a1", "a2","affs", "a1_order"]])


# for i in range(len(uni2)):
#     temp=lm[lm[order]==i]
#     lst=temp[back].unique()
#     if (lst.shape[0]==1): #check if a1 are all the same
#         continue
#     if temp.shape[0]>1: #check if there is more than a one line entry
#         duplicate=duplicate+1
#         combos=list(combinations(lst, 2))
#         for j in combos:
#             a=temp[temp[back]==j[0]]
#             b=temp[temp[back]==j[1]]
#             print(str(b.shape)+ " "+ str(a.shape))
#         # if len(lst)>2: 
#             print(len(lst))
#             print(temp[["a1", "a2","affs", "a1_order"]])
    

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                  a1           a2                     affs
8878  cliff l. lloyd  c. l. lloyd  [Khartoum Japan (City)]
                a1        a2                  affs
21506  cliff lloyd  c. lloyd   [Purdue University]
20921  cliff lloyd  c. lloyd  [University Of Iowa]
###########################
                  a1           a2                     affs
8878  cliff l. lloyd  c. l. lloyd  [Khartoum Japan (City)]
                     a1           a2                  affs
36614  cynthia b. lloyd  c. b. lloyd     [Barnard College]
35943  cynthia b. lloyd  c. b. lloyd  [Rutgers University]
###########################
                a1        a2                  affs
21506  cliff lloyd  c. lloyd   [Purdue University]
20921  cliff lloyd  c. lloyd  [University Of Iowa]
                     a1           a2                  affs
36614  cynthia b. lloyd  c. b. lloyd     [Barnard College]
35943  cynthia b. lloyd  c. b. lloyd  [Rutgers University]
###############

In [60]:
len(round1)

2013

In [125]:
len(unique_1)

4337

In [ ]:
for i in unique_0:
    proc_auth_flat_sorted[proc_auth_flat_sorted[]]

In [115]:
proc_auth_flat_sorted.a1_order.unique().shape

(16095,)

In [122]:
duplicate

2013

In [116]:
proc_auth_flat_sorted.a3_order.unique().shape

(13483,)

In [120]:
sum(proc_auth_flat_sorted['consol_id_0'].isna()==False)

26170

In [119]:
proc_auth_flat_sorted["consol_id_0"]

38172     7289
38539     None
14593     None
38043     None
37430     None
14013     None
37049     None
34024     None
34041     None
17106     8357
26205     8357
25168     8357
31443     2039
11686     2955
10899    14459
30262    14459
6095     14459
10404    14459
29126    14459
10107    14459
9454     14459
6267      None
11335     None
11367     None
11226     None
10756     None
26924     2372
27117     2372
23942     2372
23965     2372
24069     2372
33308     4262
26450     2121
27782    12456
16213     2672
4179      3011
40859     3011
40876     3011
40374     3011
20961     5072
39808    14039
8101     15455
7919     15455
10666     8914
9858      8914
9730      8914
10884     7708
27245     7708
11279     6830
30090     6830
29592     6830
29595     6830
27823     6830
27317     6830
24105     6830
9789      6830
23319     6830
22642     6830
23612     7021
2387      6168
19787     6168
2243      6168
7532      6168
12817     6168
12898     6168
35597     6168
7435      

In [110]:
sum(proc_auth_flat_sorted['consol_id_0']!=None)

41125

In [79]:
merge_pos

983

In [72]:
proc_auth_flat_sorted.head()

,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order,consol_id_0,consol_id_1,a1_first
38172,staff o. s. s. 125,s. o. s. s. 125,s. 125,125,[Harvard University],1970,https://www.jstor.org/stable/1815828,7289,12570,5277,None,None,staff
38539,henry aaron,h. aaron,h. aaron,aaron,[University Of Maryland],1967,https://www.jstor.org/stable/1814407,3705,13984,12473,None,None,henry
14593,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1909699,3705,13984,12473,None,None,henry
38043,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1818280,3705,13984,12473,None,None,henry
37430,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1974,https://www.jstor.org/stable/1816044,3705,13984,12473,None,None,henry


In [67]:
contracts

5

In [100]:
len(unique_0)

7133

In [62]:
len(unique_1)

4337

In [63]:
duplicate

2013

In [98]:
len(proc_auth_flat_sorted["a1"].unique())

16095

In [37]:
proc_auth_flat_sorted.head()

,a1,a2,a3,last,affs,year,url,a1_order,a2_order,a3_order,consol_id_0,consol_id_1
38172,staff o. s. s. 125,s. o. s. s. 125,s. 125,125,[Harvard University],1970,https://www.jstor.org/stable/1815828,7289,12570,5277,7289,None
38539,henry aaron,h. aaron,h. aaron,aaron,[University Of Maryland],1967,https://www.jstor.org/stable/1814407,3705,13984,12473,None,None
14593,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1909699,3705,13984,12473,None,None
38043,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1970,https://www.jstor.org/stable/1818280,3705,13984,12473,None,None
37430,henry aaron,h. aaron,h. aaron,aaron,"[Brookings Institution, University Of Maryland]",1974,https://www.jstor.org/stable/1816044,3705,13984,12473,None,None


In [34]:
duplicate

0

In [68]:
proc_auth_flat_sorted.to_excel(base_path+"auth_proc_w_aff_flat.xlsx", index=False)

with open(base_path+"author_proc_reduced.json", "w") as outfile: 
    json.dump(reduced, outfile, indent=4, default=int)